# Understanding Experimental Data & regression
* Modelling
    * Least squares objective function: $\sum_{i=0}^{len(observed)-1}(observed[i]-predicted[i])^2$
        * ~ variance / # experiments
    * Linear regression: find the curve of least square - polyfit
        * Assume we want to find a polynomial
        * model = pylab.polyfit(xVals, yVals, degree)
* Prediction: polyval
    * estyVals = pylab.polyval(model, xVals)
* Which fit provides more accurate estimate?
    * compare two different models for the same data: mean squared error $\frac{\sum_{i=0}^{len(observed)-1}(observed[i]-predicted[i])^2)}{len(data)}$
    * absolute goodness of fit: coefficient of determination $R^2 = 1 - \frac{\sum(y_i-p_i)^2}{\sum(y_i-\mu)^2}$, where $y_i$ are measured values, $p_i$ are predicted values, $\mu$ mean of measured values
        * capture the portion of variability in the data is accounted for by my model
        * r=1: variability is all accounted for!; r=0: the model does not capture anything
    * problem of overfitting: not only fit the underlying process, but also the noise
        * cross validation: generate model using one dataset and test them on another dataset
            * small data-set: leave-one-out cross validation
            * larger data-set
                * k-fold cross validation: partition into k equal size sets, model trained on k-1 sets, test on the remaining set
                * repeated random sampling: randomly select n elements to train model, test on the remaining elements
        * visualise as search process
        

In [ ]:
import random, pylab, numpy
def plotData(fileName):
    xVals, yVals = getData(fileName)
    xVals = pylab.array(xVals)
    yVals = pylab.array(yVals)
    xVals = xVals*9.81  #acc. due to gravity
    pylab.plot(xVals, yVals, 'bo',
               label = 'Measured displacements')

In [ ]:
def aveMeanSquareError(data, predicted):
    error = 0.0
    for i in range(len(data)):
        error += (data[i] - predicted[i])**2
    return error/len(data)

def rSquared(observed, predicted):
    error = ((predicted - observed)**2).sum()
    meanError = error/len(observed)
    return 1 - (meanError/numpy.var(observed))

In [ ]:
def genFits(xVals, yVals, degrees):
    models = []
    for d in degrees:
        model = pylab.polyfit(xVals, yVals, d)
        models.append(model)
    return models

def testFits(models, degrees, xVals, yVals, title):
    pylab.plot(xVals, yVals, 'o', label = 'Data')
    for i in range(len(models)):
        estYVals = pylab.polyval(models[i], xVals)
        error = rSquared(yVals, estYVals)
        pylab.plot(xVals, estYVals,
                   label = 'Fit of degree '\
                   + str(degrees[i])\
                   + ', R2 = ' + str(round(error, 5)))
    pylab.legend(loc = 'best')
    pylab.title(title)

In [ ]:
def LeaveOneOutCrossValidation(dataset):
    testResults = []
    for i in range(len(dataset)):
        training = dataset[:].pop(i)
        model = buildModel(training)
        testResults.append(test(model, dataset[i]))
    avg = sum(testResults)/len(testResults)

def RepeatedRandomSampling(dataset, num_trials, num_train):
    testResults = []
    for i in range(num_trials):
        trainX, trainY, testX, testY = splitData(xVals,)
        model = buildModel(training)
        testResults.append(test(model,testSet))
    avg = sum(testResults)/len(testResults)

# Machine learning
## Introduction
* basic paradigm
    * observe set of examples: training data
    * infer sth about process that generated that data
    * use inference to make predictions about previously unseen data: test data
    * variationks on paradigm:
        * supervised: given a set of feature/label pairs -> regression & classification
            * overfitting? trade-off between false positives & false negatives
        * unsupervised: given a set of feature vectors (without labels), group them into natural clusters -> clastering
* clustering and classification
    * clustering: suppose there are k different groups, but unlabeled
    * classfication: given labeled groups, WTF the subsurface in that space that separates the groups, subject to constraints on complexity of surface
        * \# of clusters
        * complexity of separating surface
        * avoid over-fitting
* Feature engineering: want to maximise signal to noise ratio (SNR)
    * Metric for feature vetors
        * Minkowski metric $dist(X1, X2, p) = (\sum_{k=1}^{len}abs(X1_k - X2_k)^p)^{p-1}$
            * usually use Euclidean metric
            * Manhattan metric may be appropriate if different dimensions are not comparable
        * scale of dimension can be important
            * z-scaling
            * linear-interpolation

In [1]:
# z-scaling
def scaleAttrs(vals):     # scale the feature by Z-scaling s.t. mean = 0, var  = 1; alternative: linear interpolation
    vals = pylab.array(vals)
    mean = sum(vals)/len(vals)
    sd = numpy.std(vals)
    vals = vals - mean
    return vals/sd

## Clustering
* An optimisation problem: Find a C that minimise dissimilarity, under constraint e.g. minimum distance between clusters/no. of clusters
    * $variability(c) = \sum_{e\in c}distance(mean(c),e)^2$ 
        * =variance * no. of clusters: why not using variance? big & bad worse than small & bad
    * $dissimilarity(C) = \sum_{c\in C}variability(c)$
    
### hierarchical clustering
* Algorithm
    1. Start by assigning each item to a cluster 
    2. Find the closest pair of clusters and merge them into a single cluster
    3. Continue until all items are clustered into a single cluster of size N. In this process, a dendogram is created. Can select number of clusters using dendogram. (agglomerative hierarchical clustering)
* Linkage metrics
    * single-linkage: shortest distance from any member of one cluster to any member of the other cluster
    * complete-linkage: greatest distance ...
    * average-linkage: average distance ...
* How is it?
    * Deterministic
    * may not be optimal: a greedy algorithm
    * flexible wrt linkage criteria
    * slow: O(n^3), O(n^2) exists for some linkage criteria
    
### K-means clustering
* most useful when you know how many clusters you want


In [ ]:
randomly chose k examples as initial centroids 
while true: 
    create k clusters by assigning each example to closest centroid
    compute k new centroids by averaging examples in each cluster
    if centroids don’t change: 
        break

* complexity: k * n * d where n is number of pts, d is time required to compute the distance between pts
* problems 
    * choosing the wrong k leads to strange results. can choose k by:
        * prior knolwedge about application domain
        * search for a good k
            * split data into training set & test set, try different k and evaluate quality of results 
            <img src = "1.png" width = "500">
                * k = 2 - a lot of positive not chosen, bad sensitivity, different characteristics not captured
                * k = 6 - total no. of positive chosen not improved
            * run hierarchical clustering on subset of data
    * results depend on initial centroids -> affect no. of iterations and final answer. can mitigate dependence by:
        * make sure they are distrubted over space
        * try multiple sets of randomly chosen initial centroids (see below)

In [ ]:
# try multiple sets of initial centroids
best = kMeans(points)
for t in range(numTrials):
    C = kMeans(points)
    if dissmilarity(C) < dissmilarity(best):
        best = C
return best

In [ ]:
def kmeans(examples, k, verbose = False):
    #Get k randomly chosen initial centroids, create cluster for each
    initialCentroids = random.sample(examples, k)
    clusters = []
    for e in initialCentroids:
        clusters.append(cluster.Cluster([e]))
        
    #Iterate until centroids do not change
    converged = False
    numIterations = 0
    while not converged:
        numIterations += 1
        #Create a list containing k distinct empty lists
        newClusters = []
        for i in range(k):
            newClusters.append([])
            
        #Associate each example with closest centroid
        for e in examples:
            #Find the centroid closest to e
            smallestDistance = e.distance(clusters[0].getCentroid())
            index = 0
            for i in range(1, k):
                distance = e.distance(clusters[i].getCentroid())
                if distance < smallestDistance:
                    smallestDistance = distance
                    index = i
            #Add e to the list of examples for appropriate cluster
            newClusters[index].append(e)
            
        for c in newClusters: #Avoid having empty clusters
            if len(c) == 0:
                raise ValueError('Empty Cluster')
        
        #Update each cluster; check if a centroid has changed
        converged = True
        for i in range(k):
            if clusters[i].update(newClusters[i]) > 0.0:
                converged = False
        if verbose:
            print('Iteration #' + str(numIterations))
            for c in clusters:
                print(c)
            print('') #add blank line
    return clusters

def trykmeans(examples, numClusters, numTrials, verbose = False):
    """Calls kmeans numTrials times and returns the result with the
          lowest dissimilarity"""
    best = kmeans(examples, numClusters, verbose)
    minDissimilarity = cluster.dissimilarity(best)
    trial = 1
    while trial < numTrials:
        try:
            clusters = kmeans(examples, numClusters, verbose)
        except ValueError:
            continue #If failed, try again
        currDissimilarity = cluster.dissimilarity(clusters)
        if currDissimilarity < minDissimilarity:
            best = clusters
            minDissimilarity = currDissimilarity
        trial += 1
    return best

## Classification
* simplest method: same as its nearest neighbour
    * problem: if you have one noisy data, you can get wrong answer
* Metrics
    * *accuracy* = (true positive + true negative) / (true positive + true negative + false positive + false negative)
    * *positive predictive value* = true positive / (true positive + false positive)  - percentage correctly identified as positive; 
    * *negative predictive value* = true negative / (true negative + false negative)  - percentage correctly identified as negative
    * *sensitivity* = true positive / (true positive + false negative) - percentage correctly found
        * e.g. screening for disease, don't want to miss one
    * *specificity* = true negative / (true negative + false positive) - percentage correctly rejected -> trade-off
        * e.g. picking ppl egligible for a heart surgery
* Testing methodology
    * leave-one-out
    * repeated random subsampling
    
### k-nearest neighbour (KNN)
* advantages
    * learning fast, no explicit training
    * no theory required
    * easy to explain
* disadvantages
    * memory-intensive, predictions can take a long time
    * no model to shed light on process that geenerated data

In [ ]:
def leaveOneOut(examples, method, toPrint = True):
    truePos, falsePos, trueNeg, falseNeg = 0, 0, 0, 0
    for i in range(len(examples)):
        testCase = examples[i]
        trainingData = examples[0:i] + examples[i+1:]
        results = method(trainingData, [testCase])
        truePos += results[0]
        falsePos += results[1]
        trueNeg += results[2]
        falseNeg += results[3]
    if toPrint:
        getStats(truePos, falsePos, trueNeg, falseNeg)
    return truePos, falsePos, trueNeg, falseNeg

def split80_20(examples):
    sampleIndices = random.sample(range(len(examples)),
                                  len(examples)//5)
    trainingSet, testSet = [], []
    for i in range(len(examples)):
        if i in sampleIndices:
            testSet.append(examples[i])
        else:
            trainingSet.append(examples[i])
    return trainingSet, testSet

def randomSplits(examples, method, numSplits, toPrint = True):
    truePos, falsePos, trueNeg, falseNeg = 0, 0, 0, 0
    random.seed(0)
    for t in range(numSplits):
        trainingSet, testSet = split80_20(examples)
        results = method(trainingSet, testSet)
        truePos += results[0]
        falsePos += results[1]
        trueNeg += results[2]
        falseNeg += results[3]
    getStats(truePos/numSplits, falsePos/numSplits,
             trueNeg/numSplits, falseNeg/numSplits, toPrint)
    return truePos/numSplits, falsePos/numSplits,\
             trueNeg/numSplits, falseNeg/numSplits

### Logistic regression
* designed for predicting probability of an event (take a finite set of values, usually 0 or 1)
* Finds weights for each feature
* Advantages
    * provides insight about the model
* Correlated features e.g. c1 + c2 + c3 = 1
    * L1 regression tends to drive one variable to zero; L2(default) regression spreads weights across variables
    * correlated features each have a small weight, but overall weight can be large. So do not overinterpret the weights
* Changing the cut-off: Receiver Operating Characteristics(ROC)
    * try different cut-off p, plot the graph and calculate 
    <img src = "2.png" width = "400">
    * 0.5 <= AUROC <= 1, green line: randomly selected, it is the worst we can do
    * look at the region in the middle

In [ ]:
import sklearn.linear_model

def buildModel(examples, toPrint = True):
    featureVecs, labels = [],[]
    for e in examples:
        featureVecs.append(e.getFeatures())
        labels.append(e.getLabel())
    LogisticRegression = sklearn.linear_model.LogisticRegression
    model = LogisticRegression().fit(featureVecs, labels)
    if toPrint:
        print('model.classes_ =', model.classes_)
        for i in range(len(model.coef_)):
            print('For label', model.classes_[1])
            for j in range(len(model.coef_[0])):
                print('   ', Passenger.featureNames[j], '=',
                      model.coef_[0][j])
    return model

def applyModel(model, testSet, label, prob = 0.5):
    testFeatureVecs = [e.getFeatures() for e in testSet]
    probs = model.predict_proba(testFeatureVecs)
    truePos, falsePos, trueNeg, falseNeg = 0, 0, 0, 0
    for i in range(len(probs)):
        if probs[i][1] > prob:    # classification
            if testSet[i].getLabel() == label:
                truePos += 1
            else:
                falsePos += 1
        else:
            if testSet[i].getLabel() != label:
                trueNeg += 1
            else:
                falseNeg += 1
    return truePos, falsePos, trueNeg, falseNeg

def lr(trainingData, testData, prob = 0.5):
    model = buildModel(trainingData, False)
    results = applyModel(model, testData, 'Survived', prob)
    return results